In [26]:
from __future__ import division

import numpy as np
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt
import imageio

import os, shutil

from util import *

In [51]:
def sample_strip(frames, directory, name, n):
    output_path = os.path.join(directory, name) + '.png'
    if os.path.exists(output_path):
        return
    step = (len(frames) - 1) // (n - 1)
    sampled_frames = [frames[i] for i in range(0, len(frames), step)]
    
    plt.figure(figsize=(15, 15))
    def subplot_frame(i, f):
        plt.subplot(1, n, i)
        if len(f.shape) == 2 or f.shape[2] == 1:
            plt.imshow(f.reshape((224, 224)), cmap='gray')
        else:
            plt.imshow(f)
        plt.axis('off')
    for i, frame in enumerate(sampled_frames):
        subplot_frame(i + 1, frame)
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()
        
def sample_frame_gif(directory, name, n):
    frames = imageio.mimread(os.path.join(directory, name) + '.gif')
    sample_strip(frames, directory, name, n)

def sample_frame_mp4(directory, name, n):
    reader = imageio.get_reader(directory + name + '.mp4')
    frames = [im for im in reader]
    sample_strip(frames, directory, name, n)
    
def sample_frames(directory, n=6):
    names = os.listdir(directory)
    for name in names:
        path = os.path.join(directory, name)
        if os.path.islink(path):
            path = os.readlink(path)
        if os.path.isdir(path):
            sample_frames(path)
        elif path.endswith('.gif'):
            sample_frame_gif(directory, name[:-4], n)
        elif path.endswith('.mp4'):
            sample_frame_mp4(directory, name[:-4], n)

In [53]:
sample_frames(Results + 'lstm/')

NeedDownloadError: Need ffmpeg exe. You can obtain it with either:
  - install using conda: conda install ffmpeg -c conda-forge
  - download by calling: imageio.plugins.ffmpeg.download()

In [22]:
sample_frames(Root + 'models')

In [45]:
selected_vids = {
    'baseball_pitch': '0050',
    'baseball_swing': '0280',
    'bench_press': '0445',
    'bowl': '0544',
    'clean_and_jerk': '0721',
    'golf_swing': '0835',
    'jump_rope': '0992',
    'jumping_jacks': '1142',
    'pullup': '1198',
    'pushup': '1370',
    'squat': '1755',
    'tennis_serve': '2319',
}

In [46]:
for subdir in 'videos_224/', 'videos_same_bbox_224/':
    input_dir = Data + subdir
    output_dir = Results
    make_dir(output_dir)
    for action, number in selected_vids.items():
        action_dir = output_dir + action + '/'
        make_dir(action_dir)
        if subdir.startswith('videos_224'):
            output_gif = action_dir + 'true_full.gif'
        else:
            output_gif = action_dir + 'true.gif'
        imageio.mimsave(output_gif, np.load(input_dir + number + '.npy'))

In [ ]:
videos_to_copy = {
    'villegas_combined_conv4/adam_l2_100_feat' : ('adam_feat', 'all'),
    'villegas_l2_baseline/default' : ('l2', 'baseball_pitch'),

    'villegas_combined/rgb_L_13_full' : ('sgd_full', 'squat'),
    
    'villegas_combined_conv2/rgb_L_13_cropped_consistent' : ('sgd_conv2', 'squat'),
    'villegas_combined/rgb_L_13_cropped_consistent' : ('sgd_conv3', 'squat'),
    'villegas_combined_conv4/rgb_L_13_cropped_consistent' : ('sgd_conv4', ['baseball_pitch', 'squat']),
    'villegas_combined_conv5/rgb_L_13_cropped_consistent' : ('sgd_conv5', 'squat'),
}

In [47]:
for subdir, (name, actions) in videos_to_copy.items():
    if actions == 'all':
        actions = list(selected_vids.keys())
    elif type(actions) == str:
        actions = [actions]
    base_dir = Models + subdir + '/test/'
    max_time_subdir = max(os.listdir(base_dir), key=lambda y: int(y[len('output-'):]))
    base_dir += max_time_subdir + '/'
    for action in actions:
        orig_file = base_dir + action + '/' + selected_vids[action] + '.gif'
        copied_file = Results + action + '/' + name + '.gif'
        shutil.copy(orig_file, copied_file)